In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

warnings.filterwarnings('ignore')

In [53]:
df = pd.read_csv('../Health.csv')
df.shape

(170761, 42)

In [3]:
# тут я выводил количество nan, оказались они только в 1 столбце(верьте мне)
df.isna().sum();

In [54]:
df[df['sagey_b'].isna()]

,hhidpn,rmstat,ragender,rahispan,raracem,riwbegy,ragey_b,sagey_b,rhltc,rhlthlm,...,rjcten,index_wave,insured_gov,uninsured,retired,collegeplus,year,cpi,rgov,sgov
40359,42633021,1,1,0,3,15apr2006,53,NaN,3,1,...,.w,8,1,0,0,0,2005,1.133913,4680.0,10032.0
48942,48544011,1,1,0,1,15oct2006,46,NaN,3,1,...,1.9,8,0,0,0,0,2005,1.133913,0.0,0.0
85784,85066011,1,1,0,2,15apr2006,68,NaN,3,0,...,.w,8,1,0,100,0,2005,1.133913,7476.0,276.0
149436,500893010,3,2,0,1,15may2006,54,NaN,4,0,...,26.7,2,0,0,0,1,2005,1.133913,0.0,0.0


In [55]:
df['sagey_b'].fillna('.r', inplace=True) # будем считать, что те 4 человека отказались от ответа

In [6]:
df.duplicated().sum()

0

In [57]:
missing_codes = ['.', '.d', '.r', '.x', '.q', '.u', '.v', '.s', '.m']
cnt_missing = 0

for col in df.columns:
    values = df[col].value_counts()

    for code in missing_codes:
        if code in values:
            cnt_missing += values[code]

cnt_missing

75238

In [58]:
print(f'Процент пропущенных: {(cnt_missing / (df.shape[0] * df.shape[1]) * 100):.2f}%')

Процент пропущенных: 1.05%


In [56]:
import datetime

months = ['jan', 'feb', 'mar', 'apr', 'may', 'jun', 'jul', 'aug', 'sep', 'oct', 'nov', 'dec']

def parse_date(el):
    day, month, year = el[:2], el[2:5], el[5:]

    return datetime.date(int(year), months.index(month) + 1, int(day))


df['riwbegy'] = df['riwbegy'].apply(parse_date)

In [14]:
df['riwbegy'] = pd.to_datetime(df['riwbegy'])
df['inter_year'] = df['riwbegy'].dt.year

In [15]:
def to_int(el):
    try:
        return int(el)
    except ValueError:
        return np.nan


def check_for_missing(el):
    try:
        _ = int(el)
        return 'OK'
    except ValueError:
        return el

In [17]:
df_without_missed_ragey_b = df[df['ragey_b'] != '.m'];
df_without_missed_ragey_b['ragey_b'] = df_without_missed_ragey_b['ragey_b'].astype(int);

median_age_by_wave = df_without_missed_ragey_b.groupby('index_wave')['ragey_b'].median();

In [18]:
def fill_missing_ragey_b(el):
    if el['ragey_b'] == '.m':
        return median_age_by_wave[el['index_wave']]
    else:
        return el['ragey_b']

df['ragey_b'] = df.apply(fill_missing_ragey_b, axis=1)
df['ragey_b'] = df['ragey_b'].astype(int)

df['year_of_birth'] = df['year'] - df['ragey_b']

In [19]:
# get year from riwbegy column
df['year_of_interview'] = df['riwbegy'].apply(lambda x: x.year)

# get year of birth
df['year_of_birth'] = df.apply(lambda x: x.year_of_interview - x.ragey_b, axis=1)

In [20]:
df['total_work_earn'] = df['riearn'] + df['siearn']
df['total_pension_earn'] = df['ripena'] + df['sipena']
df['total_gov_income'] = df['rgov'] + df['sgov']

df['total_income'] = df['total_work_earn'] + df['total_pension_earn'] + df['total_gov_income']

In [21]:
df['total_income'].describe()

count    1.707610e+05
mean     4.449124e+04
std      5.282410e+04
min      0.000000e+00
25%      1.519674e+04
50%      2.991608e+04
75%      5.718045e+04
max      6.875386e+06
Name: total_income, dtype: float64

In [22]:
# а тут меня очень смущает максимум в столбце sipena, мб это выброс
df.iloc[:, 21:25].describe()

,riearn,ripena,siearn,sipena
count,170761.000000,170761.000000,170761.000000,1.707610e+05
mean,15702.244733,3764.862189,12042.235232,2.579190e+03
std,33371.000325,12092.210896,30816.336503,2.217670e+04
min,0.000000,0.000000,0.000000,0.000000e+00
25%,0.000000,0.000000,0.000000,0.000000e+00
50%,0.000000,0.000000,0.000000,0.000000e+00
75%,21089.415692,0.000000,8000.000000,0.000000e+00
max,643233.038611,319329.942983,615098.104813,6.780589e+06


In [60]:
def get_percent_of_missed(column):
    # count where value is in missing_codes
    count_of_missing = df[column].isin(missing_codes).sum()
    return count_of_missing, count_of_missing / df.shape[0] * 100


def get_all_missing_codes(column):
    unique_values = df[column].unique()
    codes = []

    for value in unique_values:
        try:
            _ = float(value)
        except ValueError:
            codes.append(value)

    return sorted(codes)

In [61]:
print('Кол-во пропущенных значений в столбцах(остальные полные):')
for col in df.columns:
    amount, percent = get_percent_of_missed(col)

    if amount > 0:
        print(f'{col}: {amount}({percent:.2f}%), коды: {get_all_missing_codes(col)}')

Кол-во пропущенных значений в столбцах(остальные полные):
rmstat: 114(0.07%), коды: ['.m']
rahispan: 105(0.06%), коды: ['.m']
raracem: 151(0.09%), коды: ['.m']
ragey_b: 2(0.00%), коды: ['.m']
sagey_b: 55695(32.62%), коды: ['.r', '.u', '.v']
rhltc: 350(0.20%), коды: ['.d', '.m', '.p', '.r']
rhlthlm: 1108(0.65%), коды: ['.d', '.m', '.r', '.w', '.y']
rhibpe: 213(0.12%), коды: ['.d', '.m', '.r']
rdiabe: 228(0.13%), коды: ['.d', '.m', '.r']
rcancre: 314(0.18%), коды: ['.d', '.m', '.r', '.t']
rlunge: 151(0.09%), коды: ['.d', '.m', '.r']
rhearte: 197(0.12%), коды: ['.d', '.m', '.r']
rstroke: 129(0.08%), коды: ['.d', '.m', '.r']
rpsyche: 200(0.12%), коды: ['.d', '.m', '.r']
rarthre: 186(0.11%), коды: ['.d', '.m', '.r']
rhosp: 657(0.38%), коды: ['.d', '.m', '.r']
rhspnit: 1380(0.81%), коды: ['.d', '.m', '.r', '.z']
rcovr: 1357(0.79%), коды: ['.c', '.d', '.e', '.m', '.r', '.t']
rcovs: 1231(0.72%), коды: ['.c', '.d', '.e', '.m', '.r', '.t']
rjphys: 2963(1.74%), коды: ['.d', '.m', '.p', '.r', '.s'

In [63]:
df[df['sagey_b'] == '.v'].head()

,hhidpn,rmstat,ragender,rahispan,raracem,riwbegy,ragey_b,sagey_b,rhltc,rhlthlm,...,rjcten,index_wave,insured_gov,uninsured,retired,collegeplus,year,cpi,rgov,sgov
682,10644010,1,1,0,2,1996-08-15,54,.v,3,0,...,.w,3,0,0,0,0,1995,0.891173,0.0000,0.00000
705,10646020,2,2,0,2,1996-08-15,63,.v,5,0,...,.w,3,1,0,100,0,1995,0.891173,1343.6355,0.00000
1042,10956010,1,2,0,1,1994-06-08,57,.v,1,0,...,2.7,2,0,0,0,0,1993,0.852219,11879.0740,0.00000
1473,11423010,2,2,1,3,1994-06-21,61,.v,4,1,...,.w,2,1,0,100,0,1993,0.852219,7549.4922,407.14569
6644,14768010,3,1,0,1,1994-07-14,63,.v,3,1,...,.w,2,1,0,100,0,1993,0.852219,17403.4820,0.00000


In [64]:
df[df['hhidpn'] == 10644010]

,hhidpn,rmstat,ragender,rahispan,raracem,riwbegy,ragey_b,sagey_b,rhltc,rhlthlm,...,rjcten,index_wave,insured_gov,uninsured,retired,collegeplus,year,cpi,rgov,sgov
681,10644010,1,1,0,2,1992-06-08,50,42,3,0,...,.w,1,0,0,100,0,1991,0.811126,0.0,2885.3694
682,10644010,1,1,0,2,1996-08-15,54,.v,3,0,...,.w,3,0,0,0,0,1995,0.891173,0.0,0.0000
683,10644010,7,1,0,2,1994-06-18,52,.u,3,0,...,1.8,2,0,0,0,0,1993,0.852219,0.0,0.0000


фиксим пропуски в данных

In [16]:
df['ragey_b'] = df['ragey_b'].apply(to_int)
df['rhspnit'] = df['rhspnit'].apply(to_int)
df['rjweeks'] = df['rjweeks'].apply(to_int)
df['rjweek2'] = df['rjweek2'].apply(to_int)
df['rjcten'] = df['rjcten'].apply(to_int)

df['ragey_b'].fillna(df['ragey_b'].median(), inplace=True)
df['rhspnit'].fillna(df['rhspnit'].median(), inplace=True)
df['rjweeks'].fillna(df['rjweeks'].median(), inplace=True)
df['rjweek2'].fillna(df['rjweek2'].median(), inplace=True)
df['rjcten'].fillna(df['rjcten'].median(), inplace=True)

In [52]:
df[df['rmstat'] == '8']

,hhidpn,rmstat,ragender,rahispan,raracem,riwbegy,ragey_b,sagey_b,rhltc,rhlthlm,...,cpi,rgov,sgov,inter_year,year_of_birth,year_of_interview,total_work_earn,total_pension_earn,total_gov_income,total_income
27,10001010,8,1,0,1,1994-06-28,55,62.0,3,0,...,0.852219,0.0000,0.0,1994,1939,1994,15966.497149,0.0,0.0000,15966.497149
28,10001010,8,1,0,1,2012-05-15,72,62.0,3,0,...,1.306293,10416.4680,0.0,2012,1940,2012,0.000000,0.0,10416.4680,10416.468000
29,10001010,8,1,0,1,1998-03-15,58,62.0,4,0,...,0.934777,9054.0566,0.0,1998,1940,1998,0.000000,0.0,9054.0566,9054.056600
30,10001010,8,1,0,1,1996-06-15,57,62.0,3,0,...,0.891173,7191.5039,0.0,1996,1939,1996,0.000000,0.0,7191.5039,7191.503900
31,10001010,8,1,0,1,1992-10-08,53,62.0,3,0,...,0.811126,0.0000,0.0,1992,1939,1992,0.000000,0.0,0.0000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
170697,920645010,8,1,0,1,2011-09-15,56,62.0,.p,0,...,1.266227,0.0000,0.0,2011,1955,2011,34924.705507,0.0,0.0000,34924.705507
170698,920653010,8,1,0,1,2012-08-15,55,62.0,2,0,...,1.306293,0.0000,0.0,2012,1957,2012,58158.611346,0.0,0.0000,58158.611346
170699,920653010,8,1,0,1,2010-10-15,53,62.0,.p,0,...,1.245748,0.0000,0.0,2010,1957,2010,62805.646816,0.0,0.0000,62805.646816
170740,923487010,8,1,0,2,2010-10-15,53,62.0,.p,1,...,1.245748,13981.0830,0.0,2010,1957,2010,0.000000,0.0,13981.0830,13981.083000


In [ ]:
df

In [50]:
# df[df['rmstat'] == '.m']['']

,hhidpn,rmstat,ragender,rahispan,raracem,riwbegy,ragey_b,sagey_b,rhltc,rhlthlm,...,cpi,rgov,sgov,inter_year,year_of_birth,year_of_interview,total_work_earn,total_pension_earn,total_gov_income,total_income
808,10740010,.m,2,0,2,2000-08-15,62,62.0,5,1,...,0.967805,0.0000,0.0,2000,1938,2000,28119.220923,0.0,0.0000,28119.220923
1469,11423010,.m,2,1,3,1998-03-15,65,62.0,5,1,...,0.934777,9607.1982,0.0,1998,1933,1998,0.000000,0.0,9607.1982,9607.198200
1489,11466040,.m,2,0,2,1996-05-15,43,62.0,3,0,...,0.891173,13741.7270,0.0,1996,1953,1996,0.000000,0.0,13741.7270,13741.727000
4242,13296010,.m,2,1,1,2000-06-15,65,62.0,2,1,...,0.967805,7592.1895,0.0,2000,1935,2000,10544.707846,0.0,7592.1895,18136.897346
5123,13899010,.m,1,0,1,2000-05-15,62,62.0,4,0,...,0.967805,0.0000,0.0,2000,1938,2000,30462.489333,0.0,0.0000,30462.489333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
163584,529540010,.m,2,0,1,2010-09-15,56,62.0,.p,1,...,1.245748,1758.5581,0.0,2010,1954,2010,98202.253036,0.0,1758.5581,99960.811136
164539,533067010,.m,1,0,2,2011-09-15,60,62.0,.p,1,...,1.266227,4674.5376,0.0,2011,1951,2011,0.000000,0.0,4674.5376,4674.537600
166742,903171010,.m,2,0,1,2010-08-15,62,62.0,.p,1,...,1.245748,8650.7949,0.0,2010,1948,2010,0.000000,0.0,8650.7949,8650.794900
168059,908733010,.m,2,1,1,2010-06-15,52,62.0,.p,0,...,1.245748,7300.0186,0.0,2010,1958,2010,0.000000,0.0,7300.0186,7300.018600


In [171]:
df.to_csv('../Health_cleaned.csv', index=False)